In [17]:
import torch

In [18]:
torch.__version__

'2.0.0'

In [19]:
from sys import version
version

'3.10.8 (main, Nov 30 2022, 20:34:09) [Clang 14.0.0 (clang-1400.0.29.202)]'

In [20]:
# INDEXING WITH LISTS
t = torch.randn((2,5))
print(t)
print(t[1])
print(t[[0,0,1]])

# INDEXING WITH TENSORS
print(t[torch.tensor([0,1,1])])

tensor([[ 1.4783,  0.7237,  0.2772, -2.0587,  0.6789],
        [-0.4774, -0.3143,  0.5061, -0.9945, -0.1353]])
tensor([-0.4774, -0.3143,  0.5061, -0.9945, -0.1353])
tensor([[ 1.4783,  0.7237,  0.2772, -2.0587,  0.6789],
        [ 1.4783,  0.7237,  0.2772, -2.0587,  0.6789],
        [-0.4774, -0.3143,  0.5061, -0.9945, -0.1353]])
tensor([[ 1.4783,  0.7237,  0.2772, -2.0587,  0.6789],
        [-0.4774, -0.3143,  0.5061, -0.9945, -0.1353],
        [-0.4774, -0.3143,  0.5061, -0.9945, -0.1353]])


In [21]:
# INDEXING WITH TENSORS IN MULTIPLE DIMENSIONS
# https://youtu.be/TCH_1BHY58I?t=1028
x = torch.arange(12).reshape(3, 4)
print("x:\n", x)

y = torch.tensor([[0, 2], [1, 1], [2, 0]])
print("y:\n", y)

print("indexes:\n", y[:, 0], y[:, 1])
result = x[y[:, 0], y[:, 1]]
print("result:\n", result) # do you see it?

x:
 tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
y:
 tensor([[0, 2],
        [1, 1],
        [2, 0]])
indexes:
 tensor([0, 1, 2]) tensor([2, 1, 0])
result:
 tensor([2, 5, 8])


In [22]:
# more fun example from karpathy
C = torch.randn((27,2))
X = torch.arange(32*3).reshape((32,3)) % 27 # thank you chatgpt for the % 27
C.shape, X.shape, C[X].shape
# if you understand this, you can explain why the % 27 is needed

(torch.Size([27, 2]), torch.Size([32, 3]), torch.Size([32, 3, 2]))

In [23]:
# CONCATENATE
X = torch.randn((2,3))
X, torch.cat((x, x), 0), torch.cat((x, x), 1)

(tensor([[ 0.6801,  0.4981,  0.7300],
         [-0.3213, -0.0018, -1.3334]]),
 tensor([[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]),
 tensor([[ 0,  1,  2,  3,  0,  1,  2,  3],
         [ 4,  5,  6,  7,  4,  5,  6,  7],
         [ 8,  9, 10, 11,  8,  9, 10, 11]]))

In [24]:
# UNBIND
t = torch.arange(6).reshape((2,3))
t2 = torch.arange(12).reshape((2,3,2))
t, t.unbind(), t.unbind(1)

(tensor([[0, 1, 2],
         [3, 4, 5]]),
 (tensor([0, 1, 2]), tensor([3, 4, 5])),
 (tensor([0, 3]), tensor([1, 4]), tensor([2, 5])))

In [25]:
t2, t2.unbind

(tensor([[[ 0,  1],
          [ 2,  3],
          [ 4,  5]],
 
         [[ 6,  7],
          [ 8,  9],
          [10, 11]]]),
 <function Tensor.unbind>)